In [24]:
import QuantLib as ql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RBFInterpolator
from datetime import datetime, timedelta


In [25]:
spot = 100.0
r = 0.05
pricing_date = datetime(2023, 10, 2)
pricing_date_str = pricing_date.strftime('%Y-%m-%d')

vol_surface = pd.read_csv(rf'/Users/allen/Downloads/grid_data_{pricing_date_str}.csv')
vol_surface['vol'] = np.sqrt(vol_surface['total_var_grid'] / vol_surface['T_grid'])
vol_surface['fwd'] = spot * np.exp(r * vol_surface['T_grid'])
vol_surface['k'] = vol_surface['fwd'] * np.exp(vol_surface['k_grid'])

In [26]:
ks = vol_surface['k'].tolist()
ts = vol_surface['T_grid'].tolist()
xy = np.column_stack((ks / np.std(ks), ts / np.std(ts)))
xy = np.column_stack((ks, ts))
vols = vol_surface['vol'].tolist()
rbf = RBFInterpolator(xy, vols, kernel='thin_plate_spline', smoothing=1e-4)

In [27]:
dgrid = [pricing_date + timedelta(days=round(x * 365)) for x in sorted(vol_surface['T_grid'].unique())]
tgrid = [(d - pricing_date).days / 365.0 for d in dgrid]
kmin, kmax = vol_surface['k'].min(), vol_surface['k'].max()
kgrid = np.linspace(kmin, kmax, 100)

In [28]:
volmatrix = ql.Matrix(len(kgrid), len(dgrid))
for ik, k in enumerate(kgrid):
    vs = rbf([[k, t] for t in tgrid]).tolist()
    for it, v in enumerate(vs):
        volmatrix[ik][it] = v

In [29]:
dgrid = [ql.Date(d.day, d.month, d.year) for d in dgrid]

In [30]:
reference_date = ql.Date(pricing_date.day, pricing_date.month, pricing_date.year)
ql.Settings.instance().evaluationDate = reference_date
calendar = ql.TARGET()
dc = ql.Actual365Fixed()

volatility = ql.BlackVarianceSurface(
    reference_date,
    calendar,
    dgrid,
    kgrid,
    volmatrix,
    dc
)

volatility.enableExtrapolation()

In [42]:

ql.Settings.instance().evaluationDate = reference_date

exercise = ql.AmericanExercise(reference_date, ql.Date(2, ql.October, 2024))
payoff = ql.PlainVanillaPayoff(ql.Option.Put, 110.0)
option = ql.VanillaOption(payoff, exercise)

underlying = ql.SimpleQuote(100.0)
dividend = ql.FlatForward(reference_date, 0.0, ql.Actual365Fixed())
risk_free_rate = ql.FlatForward(reference_date, r, ql.Actual365Fixed())

process = ql.BlackScholesMertonProcess(
    ql.QuoteHandle(underlying),
    ql.YieldTermStructureHandle(dividend),
    ql.YieldTermStructureHandle(risk_free_rate),
    ql.BlackVolTermStructureHandle(volatility),
)

In [43]:
results = []

# option.setPricingEngine(ql.BaroneAdesiWhaleyApproximationEngine(process))
# results.append(("Barone-Adesi-Whaley", option.NPV()))

# option.setPricingEngine(ql.BjerksundStenslandApproximationEngine(process))
# results.append(("Bjerksund-Stensland", option.NPV()))

timeSteps = 801
gridPoints = 800
option.setPricingEngine(ql.FdBlackScholesVanillaEngine(process, timeSteps, gridPoints))
results.append(("finite differences", option.NPV()))

# option.setPricingEngine(ql.QdPlusAmericanEngine(process))
# results.append(("QD+", option.NPV()))

# option.setPricingEngine(
#     ql.QdFpAmericanEngine(process, ql.QdFpAmericanEngine.accurateScheme())
# )
# results.append(("QD+ fixed point", option.NPV()))

# timeSteps = 801
# for tree in ["JR", "CRR", "EQP", "Trigeorgis", "Tian", "LR", "Joshi4"]:
#     option.setPricingEngine(ql.BinomialVanillaEngine(process, tree, timeSteps))
#     results.append(("Binomial (%s)" % tree, option.NPV()))


df = pd.DataFrame(results, columns=["Method", "Option value"])
df.style.hide(axis="index")
print(df)

               Method  Option value
0  finite differences     10.256425
